In [ ]:
import geopandas as gpd
import numpy as np

In [ ]:
# Get geometry data from Natural Earth
url = "https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip"
data_v1 = gpd.read_file(url)
print(data_v1.head())

In [ ]:
# Set SRS=4326
data_v1 = data_v1.to_crs(epsg=4326)
print(data_v1.crs)

In [ ]:
# Keep only full name, ISO_A2, geometry, populatio and GDP of the table
data_v2 = data_v1[['geometry', 'NAME_EN', 'ISO_A2_EH','POP_EST','GDP_MD']]
print(data_v2)

In [ ]:
# Delete the columns without proprt ISO_A2 values
data_v3 = data_v2[data_v2['ISO_A2_EH'] != '-99']
print(data_v3)

In [ ]:
# check validity of data
validity = data_v3.is_valid

if validity.all():
    print('All geometries are valid')
# if some geometry is not valid, correct the geometry by setting the buffer
else:
    print('At least one geometry is not valid')
    # get the index where validity is False
    invalid_row_idx = np.where(validity == False)[0]
    print('Invalid row index:', invalid_row_idx)
    # locate the invalid geometries and set the buffer
    selected_rows = data_v3.iloc[invalid_row_idx] 
    selected_rows['geometry'] = selected_rows['geometry'].buffer(0)
    data_v3.iloc[invalid_row_idx] = selected_rows
    # reexmaine the validity
    validity_v2=data_v3.is_valid
    if validity_v2.all():
        print('the invalid geometry has been corrected, and all geometries are valid')
    else:
        print('Still at least one geometry is not valid')

In [ ]:
# Export the data to geopackage
data_v3.to_file('Task-2-Data/Task-2-Exported-Geomeries.gpkg', layer='ISO_A2_Geometries', driver="GPKG")